# Chapter 2. End-to-End Machine Learning Project

- the big picture

- get data

- discover and viz data

- prep data for ML algorithm

- select a model and train it

- fine-turn model

- present the solution

- lauch, monitor, and maintain the system